# Import libraries, setting global variables

In [1]:
import pandas as pd
import numpy as np
import requests
from bs4 import BeautifulSoup
import os
import json
import re

In [2]:
ENV = "colab" #or jn for work on local machine

if ENV == "colab":
  DATA_PATH = "/content/drive/MyDrive/infa/elsefiles/"
elif ENV == "jn":
  DATA_PATH = "C:/Users/danil/Desktop/infa analog/datas"

# Get models to parse

In [5]:
#brands for analysis was taken from browsing page of auto.ru
brands = pd.read_csv(os.path.join(DATA_PATH, "brands_number.csv"))

In [6]:
pd.set_option('display.max_rows', 20)

In [7]:
ru_brands = ["Lada (ВАЗ)", "УАЗ", "ГАЗ", "Москвич", "ЗАЗ", "Xcite", "Knewstar", "Tank"]

In [8]:
#We'll take only foregins brands which has more than 1000 placements
final_brands = brands[(brands['number of models'] >= 1000) & (~brands["brand"].isin(ru_brands))].sort_values('number of models', ascending=False)

In [9]:
url = "https://auto.ru/cars/all/"
t = requests.get(url)
soup = BeautifulSoup(t.content, "html.parser")

In [10]:
#Getting url that shows ads with filter on brand
brand_urls = list(map(lambda x: x["href"], filter(lambda x: x.text in final_brands["brand"].values, soup.find_all("a", "Link ListingPopularMMM__itemName"))))
brand_names = list(map(lambda x: x.text, filter(lambda x: x.text in final_brands["brand"].values, soup.find_all("a", "Link ListingPopularMMM__itemName"))))
final_brands = final_brands.merge(pd.DataFrame({"brand": brand_names, "url": brand_urls}), on="brand")

In [11]:
#Adding to fdataframe count of ads
model_urls = []
model_names = []
for brand_url in brand_urls:
  soup = BeautifulSoup(requests.get(brand_url).content, "html.parser")
  model_urls += list(map(lambda x: x["href"], soup.find_all("a", "Link ListingPopularMMM__itemName")))
  model_names += list(map(lambda x: x.text, soup.find_all("a", "Link ListingPopularMMM__itemName")))
print(len(model_urls))

1775


In [12]:
final_brands

,brand,number of models,url
0,Toyota,50503,https://auto.ru/cars/toyota/all/
1,Kia,34503,https://auto.ru/cars/kia/all/
2,BMW,31618,https://auto.ru/cars/bmw/all/
3,Hyundai,31348,https://auto.ru/cars/hyundai/all/
4,Mercedes-Benz,29566,https://auto.ru/cars/mercedes/all/
...,...,...,...
47,Jaguar,1202,https://auto.ru/cars/jaguar/all/
48,Datsun,1195,https://auto.ru/cars/datsun/all/
49,Dodge,1118,https://auto.ru/cars/dodge/all/
50,Tesla,1045,https://auto.ru/cars/tesla/all/


## Parse models

In [ ]:
#Getting all models of each brand to parse with filter on model
def parse_models(item, list_urls, list_names, list_count):
  if item.find("a", "Link ListingPopularMMM__itemName") is None or \
    item.find("div", "ListingPopularMMM__itemCount") is None:
    return None
  list_names.append(item.find("a", "Link ListingPopularMMM__itemName").text)
  list_urls.append(item.find("a", "Link ListingPopularMMM__itemName")["href"])
  list_count.append(item.find("div", "ListingPopularMMM__itemCount").text)

In [ ]:
# brand_models_dict = {}
# for brand in final_brands.iterrows():
#   brand_name = brand[1]["brand"]
#   brand_url = brand[1]["url"]
#   soup = BeautifulSoup(requests.get(brand_url).content, "html.parser")
#   raw_list = soup.find_all("div", "ListingPopularMMM__item")
#   l1, l2, l3 = [], [], []
#   for item in raw_list:
#     parse_models(item, l1, l2, l3)

#   brand_models_dict[brand_name] = pd.DataFrame({"url": l1, "model": l2, "count": l3})

In [ ]:
# df_list = []
# for brand, df in brand_models_dict.items():
#     df["brand"] = brand  # Добавляем колонку с именем бренда
#     df_list.append(df)

# final_df = pd.concat(df_list, ignore_index=True)

In [ ]:
#Code is in comments becouse it was saved in the file below
final_df = pd.read_csv(os.path.join(DATA_PATH, "brands_models_urls.csv")).drop("Unnamed: 0", axis=1)

In [ ]:
final_df

,url,model,count,brand
0,https://auto.ru/cars/toyota/avensis/used/,Avensis,1097,Toyota
1,https://auto.ru/cars/toyota/camry/all/,Camry,7915,Toyota
2,https://auto.ru/cars/toyota/corolla/all/,Corolla,5786,Toyota
3,https://auto.ru/cars/toyota/highlander/all/,Highlander,1425,Toyota
4,https://auto.ru/cars/toyota/land_cruiser/all/,Land Cruiser,3718,Toyota
...,...,...,...,...
1828,https://auto.ru/cars/faw/besturn_b30/used/,Besturn B30,2,FAW
1829,https://auto.ru/cars/faw/d60/used/,D60,1,FAW
1830,https://auto.ru/cars/faw/jinn/used/,Jinn,5,FAW
1831,https://auto.ru/cars/faw/oley/used/,Oley,12,FAW


In [ ]:
#final_df.to_csv("/content/drive/MyDrive/infa/elsefiles/brands_models_urls.csv")
#final_df.to_csv(os.path.join(DATA_PATH, "brands_models_urls.csv"))

# Functions

In [ ]:
#from item - soup of single ad on the page - getting info about km of used auto
def get_km(item):
  try:
    t = item.find("div", "ListingItem__description").find("div", "ListingItem__kmAgeAndActionsBlock").find("div", "ListingItem__kmAgeAndSalonInfo").find("div", "ListingItem__kmAge").text.replace('\xa0', '')[:-2]
    return 0 if t == "Нов" else int(t)
  except:
    print("problem get_km")
    return None

#from item - soup of single ad on the page - parsing posible tags, that could be placed right in the ad below the price
def get_salon_tag(item):
  try:
    return {
        "salon": 0 if item.find("div", "ListingItem__description").find("div", "ListingItem__kmAgeAndActionsBlock").find("div", "ListingItem__kmAgeAndSalonInfo").find("div", "ListingItemInStock-awRrm ListingItemInStock_stock-mWD02 ListingItem__stock") is None else 1,
        "seller": 0 if item.find("div", "ListingItem__description").find("div", "ListingItem__kmAgeAndActionsBlock").find("div", "ListingItem__kmAgeAndSalonInfo").find("div", "ListingItem__resellerTitle") is None else 1,
        "on order": 0 if item.find("div", "ListingItem__description").find("div", "ListingItem__kmAgeAndActionsBlock").find("div", "ListingItem__kmAgeAndSalonInfo").find("div", "ListingItemInStock-awRrm ListingItemInStock_request-YqrMT ListingItem__stock") is None else 1,
        "verified dealler": 0 if item.find("div", "ListingItem__description").find("div", "ListingItem__kmAgeAndActionsBlock").find("div", "ListingItem__kmAgeAndSalonInfo").find("div", "ListingItem__salonVerifiedLabel SalonVerifiedLabelWithPopup__popup InfoPopup InfoPopup_theme_plain InfoPopup_withChildren") is None else 1
    }
  except:
    print("problem get_salon_tag")
    return None

#from item - soup of single ad on the page - getting separate page of ad to posible further analysis
def get_url_item(item):
  try:
    return item.find("a", "Link ListingItemTitle__link")["href"]
  except:
    print("problem get_url_item")
    return None

#from item - soup of single ad on the page - getting name of ad
def get_item_name(item):
  try:
    return item.find("a", "Link ListingItemTitle__link").text
  except:
    print("problem get_item_name")
    return None

#from item - soup of single ad on the page - parsing info
def get_tags(item):
  try:
    tmp = list(map(lambda x: x.text.replace('\u2009', '').replace('\xa0', ''), item.find_all("div", "ListingItemTechSummaryDesktop__cell")))
    if "Электро" in tmp[0]:
      tags_dict = {"engine_v": float(tmp[0].split("/")[1].replace("кВт", ""))*1.35962,
                "engine_power": float(tmp[0].split("/")[0].replace("л.с.", "")),
                "fuel_type": tmp[0].split("/")[2],
                "transmission_type": tmp[1],
                "salon_type": tmp[2],
                "wheel_drive": tmp[3],
                "color": tmp[4]}
    else:
      tags_dict = {"engine_v": float(tmp[0].split("/")[0].replace("л", "")),
                  "engine_power": float(tmp[0].split("/")[1].replace("л.с.", "")),
                  "fuel_type": tmp[0].split("/")[2],
                  "transmission_type": tmp[1],
                  "salon_type": tmp[2],
                  "wheel_drive": tmp[3],
                  "color": tmp[4]}
    return tags_dict
  except:
    print("problem get_tags")
    return {"engine_v": None,
                "engine_power": None,
                "fuel_type": None,
                "transmission_type": None,
                "salon_type": None,
                "wheel_drive": None,
                "color": None}

def get_price(item):
  try:
    tmp = item.find("div", "ListingItemPrice__content")
    if tmp is None:
      return None
    else:
      t = item.find("div", "ListingItemPrice__content").text.replace('\u2009', '').replace('\xa0', '')[:-1]
      if "от" in t or "до" in t:
        t = t[3:]
      return int(t)
  except:
    print("problem get_price")
    return None

def get_year(item):
  try:
    return int(item.find("div", "ListingItem__year").text)
  except:
    print("problem get_year")
    return None

def get_region(item):
  try:
    return item.find("span", "MetroListPlace__regionName MetroListPlace_nbsp").text
  except:
    print("problem get_region")
    return None

In [ ]:
def get_count_pages(url_model_soup):
  if url_model_soup.find("span", "ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination__pages") is None:
    return 1
  return max(list(map(lambda x: 10 if x.text in ["…", "..."] else int(x.text), url_model_soup.find("span", "ControlGroup ControlGroup_responsive_no ControlGroup_size_s ListingPagination__pages").find_all("a"))))

In [ ]:
#from item - soup of single ad on the page - parsing posible tags in the bottom of ad
def get_all_info(item):
    # print(get_url_item(item))
    # print()
    # print(item)
    # print()
  result = get_tags(item)
  result["km"] = get_km(item)
  result["url"] = get_url_item(item)
  result["price"] = get_price(item)
  result["year"] = get_year(item)
  result["region"] = get_region(item)
  result["name"] = get_item_name(item)
  result["additional_tags"] = get_additional_tags(item)
  if any(result.values()) is None:
    print(result["url"])

  result.update(get_salon_tag(item))
  return result

In [ ]:
def get_additional_tags(item):
  return list(map(lambda x: x.text, item.find_all("div", "Badge")))

In [ ]:
def parse_pages(url, brand="test", model="test"):
  soup = BeautifulSoup(requests.get(url).content, "html.parser")
  pages = get_count_pages(soup)
  result = []
  for page_num in range(1, pages+1):
    current_url = f"{url}?page={page_num}"
    soup = BeautifulSoup(requests.get(current_url).content, "html.parser")
    result += parse_page(soup, brand, model, page_num)
    print(f"{brand} - {model} - {page_num} -- {len(result)}/{pages*37}")
  return result

In [ ]:
def parse_page(soup, brand="test", model="test", page="test"):
  tmp = soup.find_all("div", "ListingItem")
  result_list = list(map(get_all_info, tmp))
  result_list = [{**x, "model": model, "brand": brand, "page": page} for x in result_list]
  return result_list

To parse from each page

In [ ]:
# s = BeautifulSoup(requests.get("https://auto.ru/cars/used/sale/datsun/mi_do/1127203955-09dce813/").content)
# list(map(lambda x: x.text, s.find("li", "CardInfoRow CardInfoRow_wheel").find_all("div", "CardInfoRow__cell")))[1]
# list(map(lambda x: x.text, s.find("li", "CardInfoRow CardInfoRow_state").find_all("div", "CardInfoRow__cell")))[1]
# list(map(lambda x: x.text, s.find("li", "CardInfoRow CardInfoRow_ownersCount").find_all("div", "CardInfoRow__cell")))[1].split()[0]
# list(map(lambda x: x.text, s.find("li", "CardInfoRow CardInfoRow_pts").find_all("div", "CardInfoRow__cell")))[1]
# list(map(lambda x: x.text, s.find("li", "CardInfoRow CardInfoRow_customs").find_all("div", "CardInfoRow__cell")))[1]

In [ ]:
# date = s.find("div", "CardHead__infoItem CardHead__creationDate").text
# if len(date.split()) == 2:
#   date += " 2025"

# Test

In [ ]:
final_df[final_df["brand"] == "Toyota"].head(30)

,url,model,count,brand
0,https://auto.ru/cars/toyota/avensis/used/,Avensis,1097,Toyota
1,https://auto.ru/cars/toyota/camry/all/,Camry,7915,Toyota
2,https://auto.ru/cars/toyota/corolla/all/,Corolla,5786,Toyota
3,https://auto.ru/cars/toyota/highlander/all/,Highlander,1425,Toyota
4,https://auto.ru/cars/toyota/land_cruiser/all/,Land Cruiser,3718,Toyota
...,...,...,...,...
25,https://auto.ru/cars/toyota/blade/used/,Blade,4,Toyota
26,https://auto.ru/cars/toyota/brevis/used/,Brevis,21,Toyota
27,https://auto.ru/cars/toyota/bz3/all/,bZ3,15,Toyota
28,https://auto.ru/cars/toyota/bz4x/used/,bZ4X,24,Toyota


In [ ]:
test_url = "https://auto.ru/cars/toyota/bz3/all/"
parse_pages(test_url)[:5]

test - test - 1 -- 17/37


[{'engine_v': 183.5487,
  'engine_power': 184.0,
  'fuel_type': 'Электро',
  'transmission_type': 'Заряд на517км',
  'salon_type': 'седан',
  'wheel_drive': 'передний',
  'color': 'чёрный',
  'km': 100,
  'url': 'https://auto.ru/cars/used/sale/toyota/bz3/1127398787-1e2ba992/',
  'price': 2400070,
  'year': 2024,
  'region': 'Владивосток',
  'name': 'Toyota bZ3',
  'additional_tags': [],
  'salon': 0,
  'seller': 0,
  'on order': 1,
  'verified dealler': 0,
  'model': 'test',
  'brand': 'test',
  'page': 1},
 {'engine_v': 183.5487,
  'engine_power': 184.0,
  'fuel_type': 'Электро',
  'transmission_type': 'Заряд на517км',
  'salon_type': 'седан',
  'wheel_drive': 'передний',
  'color': 'Elite',
  'km': 0,
  'url': 'https://auto.ru/cars/new/group/toyota/bz3/23626174/23705148/1125584982-06f691a8/',
  'price': 3400000,
  'year': 2024,
  'region': 'Яблоновский',
  'name': 'Toyota bZ3',
  'additional_tags': [],
  'salon': 1,
  'seller': 0,
  'on order': 0,
  'verified dealler': 0,
  'model': 

# Parse

In [ ]:
#save_path = "C:/Users/danil/Desktop/infa analog/datas/autos/"
#saved_list = "C:/Users/danil/Desktop/infa analog/datas/autos/saved_models.txt"
save_path = os.path.join(DATA_PATH, "autos")
saved_list = os.path.join(save_path, "saved_models.txt")

In [ ]:
result = []
with open(saved_list, "r") as file:
  saved_models = [model.strip() for model in file]
for model in final_df.iterrows():
  if model[1]["model"]  in saved_models:
    model_name = model[1]['model'].replace("/", "")
    #print(model[1]["model"])
    try:
      with open(f"{save_path}{model[1]['brand']}{model_name}", "r") as outfile:
        tmp = json.load(outfile)
    except:
      print(f"{model[1]['brand']} {model_name} ERROR")
  else:
    tmp = parse_pages(model[1]["url"], model[1]["brand"], model[1]["model"])
    model_name = model[1]['model'].replace("/", "")
    with open(f"{save_path}{model[1]['brand']}{model_name}", "w") as outfile:
      json.dump(tmp, outfile)
    with open(saved_list, "a") as f:
      f.write(f"{model[1]['model']}\n")
  result += tmp

# Get saved data

In [ ]:
with open(saved_list, "r") as f:
    t = f.readline()
print(t)

Avensis



In [ ]:
with open(saved_list, "r") as file:
  saved_models = [model.strip() for model in file]

In [ ]:
saved_models[:5]

['Avensis', 'Camry', 'Corolla', 'Highlander', 'Land Cruiser']

In [ ]:
result = []
for model in final_df.iterrows():
    model_name = model[1]['model'].replace("/", "")
    try:
      with open(f"{save_path}{model[1]['brand']}{model_name}", "r") as outfile:
        tmp = json.load(outfile)
    except:
      print(f"{model[1]['brand']} {model_name} ERROR")
    result += tmp

Nissan Lafesta ERROR
Nissan R'nessa ERROR
Haval M6 ERROR
Audi Coupe ERROR
Chevrolet Cavalier ERROR
Honda e:NP1 ERROR
Honda e:NP2 ERROR
Honda e:NS1 ERROR
Honda e:NS2 ERROR
Honda Grace ERROR
Honda That'S ERROR
Mazda Proceed ERROR
Mitsubishi Delica D:2 ERROR
Mitsubishi Delica D:3 ERROR
Mitsubishi Delica D:5 ERROR
Opel Corsa ERROR
Lexus GS ERROR
Peugeot Expert ERROR
GAC M8 ERROR
Citroen GS ERROR
Citroen XM ERROR
JAC M5 ERROR
Mini Coupe ERROR
Dodge Caravan ERROR


In [ ]:
len(result)

512536

In [ ]:
#During parsing were acqiured problems with duplictated labels(Label of model have already written to file with list of models, but file brandnamemodelname does not exist)
#Decision to ignore them
final_df[final_df["model"] == "M5"]

,url,model,count,brand
256,https://auto.ru/cars/bmw/m5/all/,M5,294,BMW
1639,https://auto.ru/cars/jac/m5/used/,M5,1,JAC


In [ ]:
result[0]

{'engine_v': 1.8,
 'engine_power': 129.0,
 'fuel_type': 'Бензин',
 'transmission_type': 'механика',
 'salon_type': 'седан',
 'wheel_drive': 'передний',
 'color': 'синий',
 'km': 283000,
 'url': 'https://auto.ru/cars/used/sale/toyota/avensis/1126929489-2d46f1a7/',
 'price': None,
 'year': 2005,
 'region': 'Казань',
 'name': 'Toyota Avensis II',
 'additional_tags': ['История авто бесплатно'],
 'salon': 0,
 'seller': 0,
 'on order': 0,
 'verified dealler': 0,
 'model': 'test',
 'brand': 'Avensis',
 'page': 1}

In [ ]:
df = pd.DataFrame(result)